# MVP-V: YOLO 객체 탐지

이 노트북은 YOLOv8/v11을 사용하여 KITTI 데이터셋에서 객체 탐지를 수행합니다.

## 목표
- YOLOv8/v11 모델 로드 (사전학습 또는 KITTI fine-tuning)
- 추론 및 결과 시각화
- (선택) SAM을 활용한 pseudo-label 생성

In [ ]:
# Cell 1: 라이브러리 설치 및 임포트
import os
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import cv2
from ultralytics import YOLO
from PIL import Image

# 경로 설정
project_root = Path.cwd().parent.parent
dataset_root = project_root / "dataset" / "kitti"
print(f"프로젝트 루트: {project_root}")

In [ ]:
# Cell 2: YOLO 모델 로드
# YOLOv8 또는 YOLOv11 중 선택
model_name = 'yolov8n.pt'  # nano 버전 (빠른 추론), 'yolov8s.pt', 'yolov8m.pt', 'yolov8l.pt', 'yolov8x.pt' 사용 가능
# 또는 'yolo11n.pt' 등 최신 버전 사용 가능

model = YOLO(model_name)
print(f"모델 로드 완료: {model_name}")
print(f"모델 클래스 수: {len(model.names)}")
print(f"모델 클래스: {model.names}")

In [ ]:
# Cell 3: KITTI 데이터에 맞게 모델 Fine-tuning (선택사항)
# KITTI 데이터로 fine-tuning하려면 아래 코드 사용
# train_images_dir = dataset_root / "images" / "train"
# train_labels_dir = dataset_root / "labels" / "train"
# 
# if train_images_dir.exists():
#     results = model.train(
#         data=str(dataset_root / "kitti.yaml"),  # KITTI 데이터셋 설정 파일
#         epochs=50,
#         imgsz=640,
#         batch=16,
#         name='yolo_kitti'
#     )
# else:
#     print("훈련 데이터를 찾을 수 없습니다. 사전학습 모델을 사용합니다.")

In [ ]:
# Cell 4: 단일 이미지 추론
def predict_image(model, image_path, conf_threshold=0.25):
    """
    단일 이미지에 대한 YOLO 추론
    """
    results = model(image_path, conf=conf_threshold)
    return results[0]

# 테스트 이미지 로드
train_images_dir = dataset_root / "images" / "train"
if train_images_dir.exists():
    image_files = sorted(train_images_dir.glob("*.png"))
    if image_files:
        test_image_path = image_files[0]
        print(f"테스트 이미지: {test_image_path.name}")
        
        # 추론 실행
        results = predict_image(model, str(test_image_path))
        
        # 결과 출력
        print(f"\n탐지된 객체 수: {len(results.boxes)}")
        for i, box in enumerate(results.boxes):
            class_id = int(box.cls[0])
            conf = float(box.conf[0])
            class_name = model.names[class_id]
            print(f"  객체 {i+1}: {class_name} (신뢰도: {conf:.2f})")
    else:
        print("이미지 파일을 찾을 수 없습니다.")
else:
    print("이미지 디렉토리를 찾을 수 없습니다.")

In [ ]:
# Cell 5: 결과 시각화
def visualize_yolo_results(image_path, results, save_path=None):
    """
    YOLO 추론 결과 시각화
    """
    # 이미지 로드
    image = cv2.imread(str(image_path))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # YOLO의 내장 시각화 함수 사용
    annotated_image = results.plot()
    
    fig, axes = plt.subplots(1, 2, figsize=(20, 8))
    axes[0].imshow(image)
    axes[0].set_title("원본 이미지", fontsize=14)
    axes[0].axis('off')
    
    axes[1].imshow(annotated_image)
    axes[1].set_title("YOLO 탐지 결과", fontsize=14)
    axes[1].axis('off')
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
    
    plt.show()
    return fig

# 시각화 실행
if 'results' in locals() and 'test_image_path' in locals():
    fig = visualize_yolo_results(test_image_path, results)

In [ ]:
# Cell 6: 배치 추론 및 결과 저장
def batch_predict(model, image_dir, output_dir=None, conf_threshold=0.25):
    """
    여러 이미지에 대한 배치 추론
    """
    image_files = sorted(image_dir.glob("*.png"))
    all_results = []
    
    print(f"총 {len(image_files)}개 이미지 처리 중...")
    for i, image_path in enumerate(image_files):
        if (i + 1) % 100 == 0:
            print(f"진행: {i+1}/{len(image_files)}")
        
        results = model(str(image_path), conf=conf_threshold)
        all_results.append((image_path.name, results[0]))
    
    print(f"완료: {len(all_results)}개 이미지 처리됨")
    return all_results

# 배치 추론 실행 (선택사항 - 시간이 오래 걸릴 수 있음)
# if train_images_dir.exists():
#     batch_results = batch_predict(model, train_images_dir, conf_threshold=0.25)

In [ ]:
# Cell 7: SAM 설치 및 사용 (선택사항)
# SAM2를 사용하려면 다음 설치 필요:
# !pip install git+https://github.com/facebookresearch/segment-anything-2.git

# from sam2.build_sam import build_sam2
# from sam2.sam2_image_predictor import SAM2ImagePredictor
# 
# # SAM2 모델 로드
# sam2_checkpoint = "sam2_hiera_large.pt"  # 체크포인트 다운로드 필요
# sam2_cfg_file = "sam2_hiera_l.yaml"
# sam2 = build_sam2(sam2_cfg_file, sam2_checkpoint, device='cuda')
# sam2_predictor = SAM2ImagePredictor(sam2)
# 
# # YOLO 탐지 결과를 SAM 프롬프트로 사용
# # YOLO 박스 → SAM 세그멘테이션 마스크 생성
# def yolo_to_sam_masks(image_path, yolo_results, sam_predictor):
#     image = cv2.imread(str(image_path))
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     sam_predictor.set_image(image)
#     
#     masks = []
#     for box in yolo_results.boxes:
#         bbox = box.xyxy[0].cpu().numpy()
#         mask, scores, logits = sam_predictor.predict(box=bbox)
#         masks.append(mask)
#     
#     return masks